In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy.special import erf
from scipy.optimize import curve_fit
import os
import time

In [3]:
#Following lines are to define a couple of functions that will be used later.

def rebin_sum(a):
    if a.size%bin_ != 0:
        a = a[0:a.size//bin_ * bin_]

    return a.reshape((-1, bin_)).sum(axis=1)

def rebin_mean(a):
    if a.size%bin_ != 0:
        a = a[0:a.size//bin_ * bin_]

    return a.reshape((-1, bin_)).mean(axis=1)

# Convert energy to a number of photons counting values falling within a particular range.
# This will always return integer photon counts.
def convert_to_photon_num_range(image):
    return np.ceil(np.divide(image - photon_range[0], np.mean(photon_range)))

# Convert energy to a number of photons using the central energy of a single photon.
# This can return fractional number of photons.`
def convert_to_photon_num_mean(image):
    return image / np.mean(photon_range)

In [4]:
# Take the correct pedestal file (usually located in /sf/alvra/data/*pgroup*/res/JF_pedestals)
# Take the correct gains.h5 file (usually located in /sf/alvra/config/jungfrau/gainMaps/JF02T09V01/)

#pede_file = "/sf/alvra/data/res/p16581/pedestal_20180307_2303_res.h5"
pede_file = "/sf/alvra/data/p17803/res/JF_pedestals/pedestal_20190122_0719.JF02T09V01.res.h5"
gain_file = "/sf/alvra/config/jungfrau/gainMaps/JF02T09V01/gains.h5"

with h5py.File(gain_file, "r") as file: 
    G = file["gains"][:]
    
with h5py.File(pede_file, "r") as file: 
    P = file["gains"][:]
    mask = file["pixel_mask"][:]
    
print("Dimensions of G: ", G.shape)

channel_JFimages = "data/JF02T09V01/data"
channel_pulse_idsJF = "data/JF02T09V01/pulse_id"

channel_delay = "data/SLAAR11-LMOT-M451:ENC_1_BS/data"              #Delay of pump-probe delay stage
channel_energy = "data/SLAAR11-LTIM01-EVR0:DUMMY_PV3_NBS/data"      #Mono energy in eV
channel_pulse_idsBS = "data/SLAAR11-LMOT-M451:ENC_1_BS/pulse_id"      #Pulse ids taken from pump-probe delay stage (100 Hz)

#channel_PIPS_trans = "data/SARES11-GES1:CH1_VAL_GET/data"           #X-ray TRANS diode to PRIME Keysight (channel 1)
#channel_PIPS_fluo = "data/SARES11-GES1:CH2_VAL_GET/data"            #X-ray FLUO diode to PRIME Keysight (channel 2)
#channel_Izero = "data/SARES11-LSCP10-FNS:CH1:VAL_GET/data"          #Izero diode to PRIME Ioxos (channel 1)
#channel_Izero2 = "data/SARES11-LSCP10-FNS:CH0:VAL_GET/data"          #Izero diode to PRIME Ioxos (channel 0)
#channel_Izero3 = "data/SARES11-LSCP10-FNS:CH3:VAL_GET/data"          #Izero diode to PRIME Ioxos (channel 3)
#channel_Izero4 = "data/SARES11-LSCP10-FNS:CH2:VAL_GET/data"         #Izero diode to PRIME Ioxos (channel 2)
#channel_LaserDiode = "data/SLAAR11-LSCP1-FNS:CH0:VAL_GET/data"      #Laser diode to ESA Laser Ioxos 
#channel_Laser_refDiode = "data/SLAAR11-LSCP1-FNS:CH2:VAL_GET/data"  #Laser diode leaking from beampath, to ESA Laser Ioxos 
#channel_LAM_delaystage = "data/SLAAR11-LTIM01-EVR0:DUMMY_PV2_NBS/data"       #LAM internal stage position in mm
#channel_LAM_stepper = "data/SLAAR11-LTIM01-EVR0:DUMMY_PV10_NBS/data"         #LAM stepper motor (used for feedback)
#channel_PALM_eTOF = "data/SAROP11-PALMK118:CH2_BUFFER/data"                 #PALM eTof from Ch2
#channel_PALM_drift = "data/SAROP11-PALMK118:CH2_VAL_GET/data"               #PALM drift

Dimensions of G:  (4, 512, 9216)


In [5]:
def LoadData_JF(filename):
    with h5py.File(filename, 'r') as JF_file:
        pulse_ids = JF_file[channel_pulse_idsJF][:]
        pulse_ids = np.reshape(pulse_ids, (pulse_ids.size,))
        
        #reprate_FEL = ((pulse_ids%10 == 0) & (pulse_ids%20 != 0))              #This is for 10 Hz
        #reprate_laser = pulse_ids%20 == 0                                      #This is for 5 Hz
        reprate_FEL = ((pulse_ids%4 == 0) & (pulse_ids%8 != 0))                #This is for 25 Hz
        reprate_laser = pulse_ids%8 == 0                                       #This is for 12.5 Hz
        
        image_JF_ON = JF_file[channel_JFimages][:,:,:][reprate_laser]
        image_JF_OFF = JF_file[channel_JFimages][:,:,:][reprate_FEL]
        
        image_roi1_ON = image_JF_ON[:, roi1[0][0]:roi1[0][1], roi1[1][0]:roi1[1][1]]
        image_roi2_ON = image_JF_ON[:, roi2[0][0]:roi2[0][1], roi2[1][0]:roi2[1][1]]
        
        image_roi1_OFF = image_JF_OFF[:, roi1[0][0]:roi1[0][1], roi1[1][0]:roi1[1][1]]
        image_roi2_OFF = image_JF_OFF[:, roi2[0][0]:roi2[0][1], roi2[1][0]:roi2[1][1]]
        
    return image_roi1_ON, image_roi1_OFF, image_roi2_ON, image_roi2_OFF, pulse_ids

def LoadData_BS(filename):
    with h5py.File(filename, 'r') as BS_file:
        pulse_ids = BS_file[channel_pulse_idsBS][:]
        pulse_ids = np.reshape(pulse_ids, (pulse_ids.size,))
        
        #reprate_FEL = ((pulse_ids%10 == 0) & (pulse_ids%20 != 0))              #This is for 10 Hz
        #reprate_laser = pulse_ids%20 == 0                                      #This is for 5 Hz
        reprate_FEL = ((pulse_ids%4 == 0) & (pulse_ids%8 != 0))                #This is for 25 Hz
        reprate_laser = pulse_ids%8 == 0                                       #This is for 12.5 Hz
        
        delay = BS_file[channel_delay][:][reprate_FEL]
        energy = BS_file[channel_energy][:][reprate_FEL]
        
    return delay, energy

def errfunc(x, a, b, c, d):
    return a + b*erf((c-x)*np.sqrt(np.log(2))/(np.abs(d)))

In [6]:
#DIR = "/sf/alvra/data/p17803/raw/scan_data/CytC_Fe3_XES_timescan_7p5uJ_350nm_8000eV_014/"
#DIR = "/sf/alvra/data/p17803/raw/scan_data/CytC_Fe3_XES_timescan_5uJ_288nm_8000eV_012/"
DIR = "/sf/alvra/data/p17803/raw/"

DIR_json = "/sf/alvra/data/p17803/res/scan_info/"

#!ls -lah -t /sf/alvra/data/p17803/raw/scan_data/CytC_Fe3_XES_timescan_7p5uJ_350nm_8000eV_014/

#!ls -lah -t /sf/alvra/data/p17803/raw/scan_data/CytC_Fe3_XES_timescan_5uJ_288nm_8000eV_012/
!ls -lah -t /sf/alvra/data/p17803/raw/


!hostname

listfile = os.listdir(DIR)
number_files = len(listfile)
print ("There are",number_files,"files in the folder", DIR)

total 2.7T
drwxrws---.  7 root p17803  32K Jan 27 02:05 .
-rw-r--r--.  1 root p17803 3.3G Jan 27 02:05 XES_8kev_500fs_m2040_008.BSREAD.h5
-rw-r--r--.  1 root p17803  32M Jan 27 02:03 XES_8kev_500fs_m2040_008.BSREAD_CA.h5
-rw-r--r--.  1 root p17803  71G Jan 27 02:02 XES_8kev_500fs_m2040_008.JF02T09V01.h5
-rw-r--r--.  1 root p17803 3.3G Jan 27 01:57 XES_8kev_500fs_m1800_007.BSREAD.h5
-rw-r--r--.  1 root p17803  34M Jan 27 01:54 XES_8kev_500fs_m1800_007.BSREAD_CA.h5
-rw-r--r--.  1 root p17803  71G Jan 27 01:54 XES_8kev_500fs_m1800_007.JF02T09V01.h5
-rw-r--r--.  1 root p17803 3.3G Jan 27 01:47 XES_8kev_500fs_m1880_006.BSREAD.h5
-rw-r--r--.  1 root p17803  34M Jan 27 01:45 XES_8kev_500fs_m1880_006.BSREAD_CA.h5
-rw-r--r--.  1 root p17803  71G Jan 27 01:44 XES_8kev_500fs_m1880_006.JF02T09V01.h5
drwxrwsr-x. 97 root p17803  32K Jan 27 01:30 scan_data
-rw-r--r--.  1 root p17803 3.3G Jan 27 00:02 XES_8kev_500fs_m1880_005.BSREAD.h5
-rw-r--r--.  1 root p17803  19M Jan 27 00:00 XES_8kev_500fs_m1880_

ra-c-005.psi.ch
There are 164 files in the folder /sf/alvra/data/p17803/raw/


In [ ]:
# First we load the first images of a file to check if everything is fine with the pedestal & gain files (check highgain True or False)
# We can see also how the ROIs look like (will be useful later)

#fileName = "XES_8keV_jet_350nm_500fs_007.JF02T09V01.h5"

#fileName = "CytC_Fe3_XES_timescan_5uJ_288nm_8000eV_012_step0023.JF02T09V01.h5"

#fileName = "CytC_Fe3_XES_timescan_7p5uJ_350nm_8000eV_014_step0019.JF02T09V01.h5"
fileName = "XES_8kev_500fs_m1800_003.JF02T09V01.h5"

jf_file = h5py.File(DIR + fileName)

print(jf_file)

images = jf_file[channel_JFimages]
pulse_ids = jf_file[channel_pulse_idsJF]
print (images.shape)

nframes = 1

#nframes=images.shape(0)

total = 0
threshold_low = 2
threshold_high = 7

roi = [[220, 300], [6900, 7100]] 
#roi = [[240, 320], [7750, 7950]]
#roi = [[0, images.shape[1]], [0, images.shape[2]]]

for i in range(nframes):
    image = images[7][:]
    total += ju.apply_gain_pede(image, G=G_r_roi1, P=P_r_roi1, pixel_mask=mask_r_roi1, highgain=False)
    total[total < threshold_low] = 0
    total[total > threshold_high] = 0

total /= 1


plt.figure()
plt.imshow(total, origin="lower", extent = [roi[1][0],roi[1][-1],roi[0][0],roi[0][-1]], aspect='auto',vmax=0,vmin=8)
plt.colorbar()
plt.show()

<HDF5 file "XES_8kev_500fs_m1800_003.JF02T09V01.h5" (mode r)>
(8000, 512, 9216)


# Set the ROIs (roi1 & roi2 in pixels) in the next cell

In [7]:
# Ka emission 
roi1 = [[220, 300], [6900, 7100]] # change these numbers, here they are for both crystals1 (up & down) overlapped
#roi1 = [[7030, 7130], [220, 290]]
# Kb emission 
roi2 = [[240, 320], [7750, 7950]] # change these numbers, here they are for both crystals2 (up & down) overlapped

##################################################################

P_r_roi2 = P[:, roi2[0][0]:roi2[0][1], roi2[1][0]:roi2[1][1]]
G_r_roi2 = G[:, roi2[0][0]:roi2[0][1], roi2[1][0]:roi2[1][1]]
mask_r_roi2 = mask[roi2[0][0]:roi2[0][1], roi2[1][0]:roi2[1][1]]

P_r_roi1 = P[:, roi1[0][0]:roi1[0][1], roi1[1][0]:roi1[1][1]]
G_r_roi1 = G[:, roi1[0][0]:roi1[0][1], roi1[1][0]:roi1[1][1]]
mask_r_roi1 = mask[roi1[0][0]:roi1[0][1], roi1[1][0]:roi1[1][1]]

# The next cell is to analyse a single file or a series of identical ones to be summed up

In [ ]:
#files = ["Kalpha_Fe_foil_8keV_0.1trans_3"]          # you need to change only this name


files = ["XES_8kev_500fs_m1800_003"]
#files = ["CytC_Fe3_XES_timescan_7p5uJ_350nm_8000eV_014_step0019"]

#files = ["CytC_Fe3_XES_timescan_5uJ_288nm_8000eV_012_step0026"]


#files = ["XES_8keV_jet_288nm_500fs_005"]
#files = ["XES_8keV_jet_350nm_500fs_007"]
######################################################################################
start_time = time.time()
spectrum_ON_roi1 = 0
spectrum_OFF_roi1 = 0
spectrum_ON_roi2 = 0
spectrum_OFF_roi2 = 0
        
for f in files:
    
    JF_single_file = DIR + f + ".JF02T09V01.h5"
    print("Processing file %s" % (JF_single_file))
    
    (image_roi1_ON,image_roi1_OFF,image_roi2_ON,image_roi2_OFF,pulse_ids) = LoadData_JF(str(JF_single_file))
    
    nframes_ON = np.ma.size(image_roi1_ON,0)
    nframes_OFF = np.ma.size(image_roi1_OFF,0)
    print (nframes_ON, nframes_OFF)
    photon_range = (1, 10)
    
    for j in range(100):
        image_ON_2D_roi1 = image_roi1_ON[j,:,:]
        image_ON_2D_roi1= ju.apply_gain_pede(image_ON_2D_roi1, G=G_r_roi1, P=P_r_roi1, pixel_mask=mask_r_roi1, highgain=False)
        image_ON_2D_roi1[image_ON_2D_roi1 < threshold_low] = 0
        image_ON_2D_roi1[image_ON_2D_roi1 > threshold_high] = 0
        image_ON_2D_roi1 = convert_to_photon_num_range(image_ON_2D_roi1)
        spectrum_ON_roi1 += image_ON_2D_roi1.sum(axis=0)
        
        image_OFF_2D_roi1 = image_roi1_OFF[j,:,:]
        image_OFF_2D_roi1= ju.apply_gain_pede(image_OFF_2D_roi1, G=G_r_roi1, P=P_r_roi1, pixel_mask=mask_r_roi1, highgain=False)
        image_OFF_2D_roi1[image_OFF_2D_roi1 < threshold_low] = 0
        image_OFF_2D_roi1[image_OFF_2D_roi1 > threshold_high] = 0
        image_OFF_2D_roi1 = convert_to_photon_num_range(image_OFF_2D_roi1)
        spectrum_OFF_roi1 += image_OFF_2D_roi1.sum(axis=0)
        
        image_ON_2D_roi2 = image_roi2_ON[j,:,:]
        image_ON_2D_roi2= ju.apply_gain_pede(image_ON_2D_roi2, G=G_r_roi2, P=P_r_roi2, pixel_mask=mask_r_roi2, highgain=False)
        image_ON_2D_roi2[image_ON_2D_roi2 < threshold_low] = 0
        image_ON_2D_roi2[image_ON_2D_roi2 > threshold_high] = 0
        image_ON_2D_roi2 = convert_to_photon_num_range(image_ON_2D_roi2)
        spectrum_ON_roi2 += image_ON_2D_roi2.sum(axis=0)
        
        image_OFF_2D_roi2 = image_roi2_OFF[j,:,:]
        image_OFF_2D_roi2= ju.apply_gain_pede(image_OFF_2D_roi2, G=G_r_roi2, P=P_r_roi2, pixel_mask=mask_r_roi2, highgain=False)
        image_OFF_2D_roi2[image_OFF_2D_roi2 < threshold_low] = 0
        image_OFF_2D_roi2[image_OFF_2D_roi2 > threshold_high] = 0
        image_OFF_2D_roi2 = convert_to_photon_num_range(image_OFF_2D_roi2)
        spectrum_OFF_roi2 += image_OFF_2D_roi2.sum(axis=0)
        
print ("Job done! It took",round((time.time() - start_time),2),"seconds to process",np.size(files),"file(s)")


In [ ]:
# Let's check if everything worked out fine, plotting one spectrum for instance the spectra with laser OFF.
# This is for ROI1

x_axis = np.arange(roi1[1][0], roi1[1][1])
plt.figure()

plt.subplot(211)
plt.plot(x_axis,spectrum_OFF_roi1, label="ROI1, Spectra OFF", marker='.')
plt.legend(loc="best")
plt.title(jf_file)
x_axis = np.arange(roi2[1][0], roi2[1][1])
plt.subplot(212)
plt.plot(x_axis,spectrum_OFF_roi2,label="ROI2, Spectra OFF",marker='.')
plt.legend(loc="best")

In [ ]:
# # # This is for plotting laser on-off spectra from Roi1 to look for pump-probe

bin_ = 2    # change the bin size if wanted, 1 is every pixel

spectra_on = spectrum_ON_roi1.copy()
spectra_off = spectrum_OFF_roi1.copy()

x_axis = np.arange(roi1[1][0], roi1[1][1])

spectra_on_rebin1 = rebin_sum(spectra_on)
spectra_off_rebin1 = rebin_sum(spectra_off)
x_axis_rebin1 = rebin_mean(x_axis)

spectra_on_err1=np.sqrt(spectra_on_rebin1)
spectra_off_err1=np.sqrt(spectra_off_rebin1)

plt.figure()
plt.subplot(211)
plt.plot(x_axis_rebin1,spectra_on_rebin1, label="ON",marker='.')
plt.plot(x_axis_rebin1,spectra_off_rebin1, label="OFF",marker='.')
plt.legend(loc="best")
plt.title(jf_file)

low_err= (spectra_on_rebin1 - spectra_off_rebin1)-np.sqrt(spectra_on_err1**2+spectra_off_err1**2)
high_err= (spectra_on_rebin1 - spectra_off_rebin1)+np.sqrt(spectra_on_err1**2+spectra_off_err1**2)

plt.subplot(212)
plt.plot(x_axis_rebin1,spectra_on_rebin1 - spectra_off_rebin1, label="ON - OFF",marker='.')
plt.legend(loc="best")
plt.fill_between(x_axis_rebin1, low_err, high_err, color='lightblue')
plt.show()

In [ ]:
# # # This is for plotting laser on-off spectra from Roi2 to look for pump-probe

bin_ = 2     # change the bin size if wanted, 1 is every pixel

spectra_on = spectrum_ON_roi2.copy()
spectra_off = spectrum_OFF_roi2.copy()

x_axis = np.arange(roi2[1][0], roi2[1][1])

spectra_on_rebin2 = rebin_sum(spectra_on)
spectra_off_rebin2 = rebin_sum(spectra_off)
x_axis_rebin2 = rebin_mean(x_axis)

spectra_on_err2=np.sqrt(spectra_on_rebin2)
spectra_off_err2=np.sqrt(spectra_off_rebin2)
plt.figure()
plt.subplot(211)
plt.plot(x_axis_rebin2,spectra_on_rebin2, label="ON",marker='.')
plt.plot(x_axis_rebin2,spectra_off_rebin2, label="OFF",marker='.')
plt.legend(loc="best")
plt.title(jf_file)

low_err= (spectra_on_rebin2 - spectra_off_rebin2)-np.sqrt(spectra_on_err2**2+spectra_off_err2**2)
high_err= (spectra_on_rebin2 - spectra_off_rebin2)+np.sqrt(spectra_on_err2**2+spectra_off_err2**2)

plt.subplot(212)
plt.plot(x_axis_rebin2,spectra_on_rebin2 - spectra_off_rebin2, label="ON - OFF",marker='.')
plt.legend(loc="best")
plt.fill_between(x_axis_rebin2, low_err, high_err, color='lightblue')
plt.show()

# The next cell is for looping over several files of a scan.

In [ ]:
#difference_roi1_test = spectrum_ON_roi1[:,6950:7030:1].sum() - spectrum_OFF_roi1[:,6950:7030:1].sum()
#print(differece_roi1_test.shape)
difference_roi1_test = spectrum_ON_roi1[50:150:1] - spectrum_OFF_roi1[50:150:1]
test= np.abs(difference_roi1_test).sum()

print(difference_roi1_test)
print(test)


spectest = image_ON_2D_roi1
print(image_ON_2D_roi1.shape)
plt.figure()
plt.imshow(image_ON_2D_roi1[:,50:150:1])



In [ ]:
scan_name ="CytC_Fe3_XES_timescan_5uJ_288nm_8000eV_010"          # you need to change only this name

######################################################################################
start_time = time.time()
intermediate_time = time.time()
json_file = DIR_json + scan_name + "_scan_info.json"
print (json_file)

with open(json_file) as file:
    data = json.load(file)
numFiles = len(data['scan_files'])
#numFiles = 1
looptime0 = time.time()
print ("Processing",numFiles,"files")

threshold = 2
photon_range = (1, 3)

Energy_eV = []
Delay_mm = []
XAS_trans_roi1 = []
Error_XAS_roi1 = []
XAS_trans_roi2 = []
Error_XAS_roi2 = []
RIXS_transient_roi1 = []
RIXS_transient_roi2 = []
Error_transient_roi1 = []
Error_transient_roi2 = []
RIXS_unpumped_roi1 = []
RIXS_unpumped_roi2 = []
Error_unpumped_roi1 = []
Error_unpumped_roi2 = []
RIXS_pumped_roi1 = []
RIXS_pumped_roi2 = []
Error_pumped_roi1 = []
Error_pumped_roi2 = []

for i in range(0,numFiles):
    print("step",i+1,"of",numFiles,": Processing %s" % (str(data['scan_files'][i][1])))
    
    spectrum_ON_roi1 = 0
    spectrum_OFF_roi1 = 0
    spectrum_ON_roi2 = 0
    spectrum_OFF_roi2 = 0

    (delay_loop, energy_loop) = LoadData_BS(str(data['scan_files'][i][0]))    
    looptime1 = time.time() - looptime0
    print("t1 = " + str(looptime1) + "  time after load data BS ")
    
    energy_loop = [x for x in energy_loop if (x > 0)]     #This is to prevent that some occasional zeros screw up the mean value calculated below
    delay_loop = [x for x in delay_loop if (x > 0)]       #This is to prevent that some occasional zeros screw up the mean value calculated below
    Delay_mm.append(np.mean(delay_loop))
    Energy_eV.append(np.mean(energy_loop))
    
    (image_ON_single_roi1,image_OFF_single_roi1,image_ON_single_roi2,image_OFF_single_roi2,pulse_ids) = LoadData_JF(str(data['scan_files'][i][1]))
    looptime2 = time.time() - looptime0
    print("t2 = " + str(looptime2)+ "  time after load data JF")  
    
    nframes_ON = np.ma.size(image_ON_single_roi1,0)
    nframes_OFF = np.ma.size(image_OFF_single_roi1,0)
    print (nframes_ON, nframes_OFF)
    
    for j in range(nframes_ON):
        image_ON_2D_roi1 = image_ON_single_roi1[j,:,:]
        image_ON_2D_roi1= ju.apply_gain_pede(image_ON_2D_roi1, G=G_r_roi1, P=P_r_roi1, pixel_mask=mask_r_roi1, highgain=False)  
        image_ON_2D_roi1[image_ON_2D_roi1 < threshold] = 0
        image_ON_2D_roi1 = convert_to_photon_num_range(image_ON_2D_roi1)
        spectrum_ON_roi1 += image_ON_2D_roi1.sum(axis=0)
        
        image_OFF_2D_roi1 = image_OFF_single_roi1[j,:,:]
        image_OFF_2D_roi1= ju.apply_gain_pede(image_OFF_2D_roi1, G=G_r_roi1, P=P_r_roi1, pixel_mask=mask_r_roi1, highgain=False)
        image_OFF_2D_roi1[image_OFF_2D_roi1 < threshold] = 0
        image_OFF_2D_roi1 = convert_to_photon_num_range(image_OFF_2D_roi1)
        spectrum_OFF_roi1 += image_OFF_2D_roi1.sum(axis=0)
        
        image_ON_2D_roi2 = image_ON_single_roi2[j,:,:]
        image_ON_2D_roi2= ju.apply_gain_pede(image_ON_2D_roi2, G=G_r_roi2, P=P_r_roi2, pixel_mask=mask_r_roi2, highgain=False)
        image_ON_2D_roi2[image_ON_2D_roi2 < threshold] = 0
        image_ON_2D_roi2 = convert_to_photon_num_range(image_ON_2D_roi2)
        spectrum_ON_roi2 += image_ON_2D_roi2.sum(axis=0)
        
        image_OFF_2D_roi2 = image_OFF_single_roi2[j,:,:]
        image_OFF_2D_roi2= ju.apply_gain_pede(image_OFF_2D_roi2, G=G_r_roi2, P=P_r_roi2, pixel_mask=mask_r_roi2, highgain=False)
        image_OFF_2D_roi2[image_OFF_2D_roi2 < threshold] = 0
        image_OFF_2D_roi2 = convert_to_photon_num_range(image_OFF_2D_roi2)
        spectrum_OFF_roi2 += image_OFF_2D_roi2.sum(axis=0)
    
    #difference_roi1 = spectrum_ON_roi1.sum() - spectrum_OFF_roi1.sum()
    spectrum_ON_roi1_baseline = spectrum_ON_roi1[50:150:1] - spectrum_ON_roi1[50:150:1][0:5:1].mean()
    spectrum_OFF_roi1_baseline = spectrum_OFF_roi1[50:150:1] - spectrum_OFF_roi1[50:150:1][0:5:1].mean()
    difference_roi1 = (spectrum_ON_roi1_baseline - spectrum_OFF_roi1_baseline)
    difference_roi1 = np.abs(difference_roi1).sum()
    
    XAS_trans_roi1.append(difference_roi1)
    error_ON_roi1 = np.sum(np.sqrt(spectrum_ON_roi1))
    error_OFF_roi1 = np.sum(np.sqrt(spectrum_OFF_roi1))
    

    Error_XAS_roi1.append(np.sqrt(error_ON_roi1**2 + error_OFF_roi1**2))
    RIXS_unpumped_roi1.append(spectrum_OFF_roi1)
    Error_unpumped_roi1.append(np.sqrt(spectrum_OFF_roi1))
    RIXS_pumped_roi1.append(spectrum_ON_roi1)
    Error_pumped_roi1.append(np.sqrt(spectrum_ON_roi1))
    RIXS_transient_roi1.append(spectrum_ON_roi1 - spectrum_OFF_roi1)
    Error_transient_roi1.append(np.sqrt(np.sqrt(spectrum_ON_roi1)**2 + np.sqrt(spectrum_OFF_roi1)**2))
    
    difference_roi2 = spectrum_ON_roi2.sum() - spectrum_OFF_roi2.sum()
    XAS_trans_roi2.append(difference_roi2)
    error_ON_roi2 = np.sum(np.sqrt(spectrum_ON_roi2))
    error_OFF_roi2 = np.sum(np.sqrt(spectrum_OFF_roi2))
    
    Error_XAS_roi2.append(np.sqrt(error_ON_roi2**2 + error_OFF_roi2**2))
    RIXS_unpumped_roi2.append(spectrum_OFF_roi2)
    Error_unpumped_roi2.append(np.sqrt(spectrum_OFF_roi2))
    RIXS_pumped_roi2.append(spectrum_ON_roi2)
    Error_pumped_roi2.append(np.sqrt(spectrum_ON_roi2))
    RIXS_transient_roi2.append(difference_roi2)
    Error_transient_roi2.append(np.sqrt(np.sqrt(spectrum_ON_roi2)**2 + np.sqrt(spectrum_OFF_roi2)**2))
    
    print ("It took",round((time.time() - intermediate_time),2),"s to process this file")
    intermediate_time = time.time()
    
Energy_eV = np.array(Energy_eV)
Delay_mm = np.array(Delay_mm)

XAS_trans_roi1 = np.array(XAS_trans_roi1)
Error_XAS_roi1 = np.array(Error_XAS_roi1)
RIXS_unpumped_roi1 = np.array(RIXS_unpumped_roi1)
Error_unpumped_roi1 = np.array(Error_unpumped_roi1)
RIXS_pumped_roi1 = np.array(RIXS_pumped_roi1)
Error_pumped_roi1 = np.array(Error_pumped_roi1)
RIXS_transient_roi1 = np.array(RIXS_transient_roi1)
Error_transient_roi1 = np.array(Error_transient_roi1)

XAS_trans_roi2 = np.array(XAS_trans_roi2)
Error_XAS_roi2 = np.array(Error_XAS_roi2)
RIXS_unpumped_roi2 = np.array(RIXS_unpumped_roi2)
Error_unpumped_roi2 = np.array(Error_unpumped_roi2)
RIXS_pumped_roi2 = np.array(RIXS_pumped_roi2)
Error_pumped_roi2 = np.array(Error_pumped_roi2)
RIXS_transient_roi2 = np.array(RIXS_transient_roi2)
Error_transient_roi2 = np.array(Error_transient_roi2)

print ("Job done! It took",round((time.time() - start_time),2),"seconds to process",numFiles,"file(s)")

In [ ]:
error_high = XAS_trans_roi1 + Error_XAS_roi1
error_low = XAS_trans_roi1 - Error_XAS_roi1

plt.figure(figsize=(9,5))
plt.subplots_adjust(wspace=0.5)

plt.subplot(121)
plt.title("XAS")
#plt.plot(Energy_eV, XAS_trans_roi1,label="Energy scan",marker='.') #This is the meaningful plot if it is a monoscan
#plt.fill_between(np.squeeze(Energy_eV), error_low, error_high, color='lightblue')
plt.subplot(122)
plt.title("Delay scan")
plt.plot(Delay_mm, XAS_trans_roi1,label="Delay scan",marker='.') #This is the meaningful plot if it is a delay scan
#plt.fill_between(np.squeeze(Delay_mm), error_low, error_high, color='lightblue')


In [ ]:
#extent = [roi[1][0],roi[1][-1],roi[0][0],roi[0][-1]]

plt.figure(figsize=(5,9))
plt.subplots_adjust(hspace=0.5)

plt.subplot(311)
plt.imshow(RIXS_unpumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower")
plt.title("RIXS unpumped")
plt.colorbar()
plt.show()

plt.subplot(312)
plt.imshow(RIXS_pumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower")#, vmin=-250, vmax=250)
plt.title("RIXS pumped")
plt.colorbar()
plt.show()

plt.subplot(313)
plt.imshow(RIXS_transient_roi1, aspect = 'auto', cmap = "hot", origin ="lower", vmin=-50, vmax=50)
plt.title("RIXS transient")
plt.colorbar()
plt.show()

#fig.savefig('RIXS_unpumped')

In [ ]:
#### This is to save h5 data ####

savedir = "/das/work/p17/p17803/Processed_files/"

h5f = h5py.File(savedir + 'CytC_Fe3_monoscan_288nm_5uJ_500fs.h5','w') 
#h5f = h5py.File(savedir + 'timeScan_4.h5','w')     
#h5f = h5py.File(savedir + 'RIXS_144.h5','w')

h5f.create_dataset('Delay_mm_BS', data = Delay_mm)
h5f.create_dataset('Energy_BS', data = Energy_eV)
h5f.create_dataset('XAS_transient', data = XAS_trans_roi1)
h5f.create_dataset('Err_XAS_transient', data = Error_XAS_roi1)
h5f.create_dataset('RIXS_unpumped', data = RIXS_unpumped_roi1)
h5f.create_dataset('Err_RIXS_unpumped', data = Error_unpumped_roi1)
h5f.create_dataset('RIXS_transient', data = RIXS_transient_roi1)
h5f.create_dataset('Err_RIXS_transient', data = Error_transient_roi1)

h5f.close()